In [5]:
#####可変基底非負値テンソル分解#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import gc
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [6]:
####データの生成####
##データと日付idの設定
#データの設定
k = 10
k_vec = np.repeat(1.0, k)
mode = 3
m = 3
hh = 2500
location = 1500
week = 7
month = 31
year = 12
max_week = 5
month_days = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
year_days = np.sum(month_days)

#日付IDを設定
day_id = np.tile(np.arange(week), int((year_days*m)/week) + 1)[:m*year_days]
week_id = np.tile(np.array(list(itertools.chain(*[np.repeat(np.arange(max_week), week)[:month_days[j]] for j in range(year)]))), m)
month_id = np.tile(np.repeat(np.arange(year), month_days), m)
date_id = [day_id, week_id, month_id]
date_n = np.array([week, max_week, year])

In [7]:
##パラメータの生成
#事前分布の設定
gamma = np.array([3.0, 1.0, 0.25, 0.5])
alpha = np.array([0.25])
beta = np.array([0.75])

#モデルパラメータの生成
omega = np.random.dirichlet(gamma, hh)
theta_u = np.random.gamma(alpha, 1/beta, hh*k).reshape(hh, k)
theta_v = np.random.gamma(alpha, 1/beta, location*k).reshape(location, k)
theta_d = [j for j in range(m+1)]
for j in range(mode):
    theta_d[j] = np.random.gamma(alpha, 1/beta, date_n[j]*k).reshape(date_n[j], k)
theta_d[mode] = np.repeat(1.0, k)

In [8]:
##応答変数を生成
#データの設定
n = location*m*year_days
freq_par = np.random.gamma(30.0, 1/0.15, hh)
location_pattern = np.tile(np.arange(location), m*year_days)
date_pattern = [np.repeat(date_id[j], location) for j in range(m)]

#結合パラメータを定義
joint_theta = [j for j in range(m+1)]
for j in range(mode+1):
    if j < mode:
        joint_theta[j] = theta_v[location_pattern, ] * theta_d[j][date_pattern[j], ]
    else:
        joint_theta[j] = theta_v[location_pattern, ]

#データの格納用配列
z_list = [i for i in range(hh)]
y_list = [i for i in range(hh)]
d_list = [i for i in range(hh)]
location_list = [i for i in range(hh)]
day_list = [i for i in range(hh)]
week_list = [i for i in range(hh)]
month_list = [i for i in range(hh)]
pt = np.repeat(0, hh)

for i in range(hh):
    if i%100==0:
        print(i)
        
    #全パターンの頻度を生成
    z = np.random.multinomial(1, omega[i, ], n)
    Lambda = np.zeros((n, mode+1))
    for j in range(mode+1):
        Lambda[:, j] = np.dot(theta_u[i, ] * joint_theta[j], k_vec)
    y_freq = np.random.poisson(np.sum(z * Lambda, axis=1))
    
    #データをサンプリング
    index_nonzero = np.where(y_freq > 0)[0]
    index_freq = np.where(np.random.binomial(1, freq_par[i]/index_nonzero.shape[0], index_nonzero.shape[0]))[0]
    index_target = index_nonzero[index_freq]

    #リストに格納
    z_list[i] = z[index_target, ]
    y_list[i] = y_freq[index_target]
    d_list[i] = np.repeat(i, index_target.shape[0])
    location_list[i] = location_pattern[index_target]
    day_list[i] = date_pattern[0][index_target]
    week_list[i] = date_pattern[1][index_target]
    month_list[i] = date_pattern[2][index_target]
    pt[i] = index_target.shape[0]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [15]:
#リストを変換
z = np.array(list(itertools.chain(*[z_list[i] for i in range(hh)])))
y = np.array(list(itertools.chain(*[y_list[i] for i in range(hh)])))
d_id = np.array(list(itertools.chain(*[d_list[i] for i in range(hh)])))
location_id = np.array(list(itertools.chain(*[location_list[i] for i in range(hh)])))
day_id = np.array(list(itertools.chain(*[day_list[i] for i in range(hh)])))
week_id = np.array(list(itertools.chain(*[week_list[i] for i in range(hh)])))
month_id = np.array(list(itertools.chain(*[month_list[i] for i in range(hh)])))
date_id = [day_id, week_id, month_id]
N = y.shape[0]

IndexError: list index out of range

In [13]:
#インデックスを作成
d_list = [i for i in range(hh)]
location_list = [i for i in range(location)]
date_list = [i for i in range(m)]
location_n = np.repeat(0, location)
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")
for j in range(location):
    location_list[j] = np.array(np.where(location_id==j)[0], dtype="int")
    location_n[j] = location_list[j].shape[0]
for i in range(mode):
    temp_list = [j for j in range(date_n[i])]
    for j in range(date_n[i]):
        temp_list[j] = np.array(np.where(date_id[i]==j)[0], dtype="int")
    date_list[i] = temp_list

array([592, 249, 296, ..., 254, 145, 148])

In [10]:
date_n

array([ 7,  5, 12])

array([221.39797457])